In [21]:
%reload_kedro

2023-03-07 14:57:59,760 - kedro.extras.extensions.ipython - INFO - No path argument was provided. Using: /Users/guilhermepalazzo/Desktop/dev/git_repositories/crypto_msc_thesis
2023-03-07 14:58:00,081 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2023-03-07 14:58:00,275 - kedro.extras.extensions.ipython - INFO - ** Kedro project crypto_thesis
2023-03-07 14:58:00,276 - kedro.extras.extensions.ipython - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`


In [10]:
# target: LTCUSDT
catalog.load(name="portfolio_pnl", version="2023-03-07T12.13.55.024Z").operation_profit.sum()

2023-03-07 09:54:31,874 - kedro.io.data_catalog - INFO - Loading data from `portfolio_pnl` (ParquetDataSet)...


12.810000000000151

In [9]:
# target: BTCUSDT
catalog.load(name="portfolio_pnl", version="2023-03-02T18.32.09.372Z").operation_profit.sum()

2023-03-07 09:54:13,916 - kedro.io.data_catalog - INFO - Loading data from `portfolio_pnl` (ParquetDataSet)...


1689.4099999999817

In [13]:
# target: ETHUSDT
catalog.load("portfolio_pnl", version="2023-03-07T12.48.56.678Z").operation_profit.sum()

2023-03-07 10:30:23,511 - kedro.io.data_catalog - INFO - Loading data from `portfolio_pnl` (ParquetDataSet)...


-25.879999999998972

In [14]:
# target: BTCUSDT
catalog.load(name="portfolio_pnl", version="2023-03-07T13.29.13.279Z").operation_profit.sum()

2023-03-07 10:34:39,749 - kedro.io.data_catalog - INFO - Loading data from `portfolio_pnl` (ParquetDataSet)...


2372.6699999999782

In [22]:
df = catalog.load("xgboost_model_reporting")

2023-03-07 14:58:02,827 - kedro.io.data_catalog - INFO - Loading data from `xgboost_model_reporting` (ParquetDataSet)...


In [23]:
df

,accuracy,model_params,data_interval,probas,fte_importance,target_name,volume_bar_size,bar_ahead_predict,labeling_tau,train_test_cutoff_date
0,0.633413,"{'use_label_encoder': False, 'eval_metric': 'm...",15m,"{1284: {'proba_label_0': 0.9995845556259155, '...","{'log_return_accum__ETHUSDT': 205.0, 'log_retu...",BTCUSDT,50000,1,0.05,2022-07-01
